In [16]:
#1. IMPORTING INITIAL NECESSARY LIBRARIES
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
import matplotlib.pyplot as plt

In [17]:
#2. DATA PREPERATION / PREPROCESSING

#2.1 Importing the dataset:
with np.load('train_data_label.npz') as data:
    X_train = data['train_data']
    Y_train = data['train_label']

with np.load('test_data_label.npz') as data:
    X_test = data['test_data']
    Y_test = data['test_label']

In [18]:
#2.3 Splitting the training data set into a training and validation set:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.25, random_state = 999)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape)

#2.4 Reshaping the data:
X_train = X_train.reshape(-1, 28,28, 1)
print(X_train.shape)

X_val = X_val.reshape(-1, 28, 28, 1)
print(X_val.shape)

X_test = X_test.reshape(-1, 28, 28, 1)
print(X_test.shape)

#2.5 Converting the labels to binary form:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

Y_train = lb.fit_transform(Y_train)
Y_val = lb.fit_transform(Y_val)
Y_test = lb.fit_transform(Y_test)


(20591, 784) (20591,) (6864, 784) (6864,) (7172, 784) (7172,)
(20591, 28, 28, 1)
(6864, 28, 28, 1)
(7172, 28, 28, 1)


In [19]:
#3. BUILDING THE CNN MODEL

#3.1 Convolution layers:
model = Sequential()
model.add(Conv2D(128, kernel_size = (5, 5),
                 strides = 1, padding = 'same', activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPool2D(pool_size = (3, 3), strides = 2, padding = 'same'))
model.add(Conv2D(64, kernel_size = (2, 2),
                strides = 1, activation = 'relu', padding = 'same'))
model.add(MaxPool2D((2, 2), 2, padding = 'same'))
model.add(Conv2D(32, kernel_size = (2, 2),
                strides = 1,activation = 'relu', padding = 'same'))
model.add(MaxPool2D((2, 2), 2, padding = 'same'))
          
model.add(Flatten())

In [20]:
#3.2 Dense and output layers:
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(rate = 0.25))

model.add(Dense(units = 24, activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 128)       3328      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        32832     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 32)          8224      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)              

In [21]:
#3.3 Adding the optimizer, loss and metric:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [22]:
#4. TRAINING THE CNN MODEL
history = model.fit(X_train, Y_train, batch_size = 25,
                    epochs = 26,
                    verbose = 2,
                    validation_data = (X_val, Y_val)
                               )

Epoch 1/26
824/824 - 37s - loss: 1.6748 - accuracy: 0.5028 - val_loss: 0.6262 - val_accuracy: 0.7893
Epoch 2/26
824/824 - 42s - loss: 0.3446 - accuracy: 0.8804 - val_loss: 0.1100 - val_accuracy: 0.9665
Epoch 3/26
824/824 - 36s - loss: 0.1503 - accuracy: 0.9504 - val_loss: 0.0475 - val_accuracy: 0.9837
Epoch 4/26
824/824 - 38s - loss: 0.1208 - accuracy: 0.9603 - val_loss: 0.0237 - val_accuracy: 0.9924
Epoch 5/26
824/824 - 40s - loss: 0.1204 - accuracy: 0.9626 - val_loss: 0.0612 - val_accuracy: 0.9811
Epoch 6/26
824/824 - 35s - loss: 0.1096 - accuracy: 0.9667 - val_loss: 0.0886 - val_accuracy: 0.9698
Epoch 7/26
824/824 - 36s - loss: 0.1000 - accuracy: 0.9716 - val_loss: 0.0266 - val_accuracy: 0.9910
Epoch 8/26
824/824 - 38s - loss: 0.0678 - accuracy: 0.9790 - val_loss: 0.0227 - val_accuracy: 0.9921
Epoch 9/26
824/824 - 39s - loss: 0.0982 - accuracy: 0.9723 - val_loss: 0.0150 - val_accuracy: 0.9964
Epoch 10/26
824/824 - 37s - loss: 0.0810 - accuracy: 0.9783 - val_loss: 0.0286 - val_accura

In [66]:
# Prediction
def predict_image(img, model):
    classes = lb.classes_
    X = img.reshape((28*28, )).reshape(-1, 28, 28, 1)
    Y_pred = model.predict(X, batch_size = 25)
    Y_pred_class_index = np.argmax(Y_pred, axis = 1)
    Y_pred_class = np.take(classes, Y_pred_class_index)
    return Y_pred_class[0]
predict_image(i4, model)

24

In [69]:
predict_data = np.load('test_images_task2.npy')
all_prediction = []
for i in range (predict_data.shape[0]):
    # step 1: delete background and split each image
    
    long_img = predict_data[i].astype("int")   #select each image
    mode, count = stats.mode(long_img, axis = 0)   # the mode of background is 200
    condition = (mode == 200)
    index = np.where(np.any(condition == True, axis=0))[0]    # find the index of the 200
    delete_noise = np.delete(long_img, index, axis=1)         # delete all the 200
    if delete_noise.shape[1] % 28 == 0:                    # check if the long image can be splitted into images with width of 28
        n_pic = int(delete_noise.shape[1]/28)                 # the number of pics in one original image
        splitted = np.array(np.hsplit(delete_noise, n_pic))   # split the long image into n_pic images, store in an array
    else:
        row_not_fit = round(delete_noise.shape[1]/28) * 28 - delete_noise.shape[1]
        n_pic = round(delete_noise.shape[1]/28)
        if row_not_fit > 0:
            to_add = np.full((28, row_not_fit), 255)
            delete_noise = np.append(delete_noise, to_add, axis=1)
            splitted = np.array(np.hsplit(delete_noise, n_pic))
        if row_not_fit < 0:
            delete_noise = delete_noise[:, :-row_not_fit]
            splitted = np.array(np.hsplit(delete_noise, n_pic))

    # step 2: do prediction on each image in "splitted", and save the result in one array for each long image
    each_image = ""
    for img in splitted:
        # (note: i transfer to tensor data, but please use your cnn way to do this step)
        prediction = predict_image(img, model)    #also put your cnn model here to predict
        each_image += str(prediction)
    all_prediction.append(each_image)
    
print(all_prediction[:10])

['19181012', '2423142413', '3201617', '22318819', '21214', '2007717', '121523205', '019220', '158615', '33112211']


In [80]:
#all_prediction = np.array(all_prediction)
CNN_prediction = pd.DataFrame(all_prediction)
CNN_prediction.to_csv("CNN_prediction.csv")